# The epidemiology of antibiotic resistance in hospitals: Paradoxes and prescriptions
Marc Lipsitch, Carl Bergstrom and Bruce Levin

In [2]:
import pandas as pd
import numpy as np
import os

from scipy.interpolate import UnivariateSpline


In [ ]:

def binomial_transition(var, rate, dt=1):
    kb = np.maximum(1.0 - np.exp(-rate * dt), 0)
    num_ind   = np.random.binomial(list(var), kb )

    return num_ind

def deterministic_transition(var, rate, dt=1):

    return var * rate * dt

def model(x,  beta, num_ensembles=300, dt=1, transition=binomial_transition):
    """[summary]

    Args:
        x ([type]):     State variables                            | Dim: [num_variables, num_age_groups, num_ensembles]
        beta ([type]):  Population level contact rate              | Dim: [num_ensembles]
        ifr ([type]):   Age group dependent infect fatality rates. | Dim: [num_age_groups, num_ensembles]
        alpha ([type]): Age group dependent ascertainment rates.   | Dim: [num_age_groups, num_ensembles]
        N ([type]):     Population in each age group.              | Dim: [num_age_groups]

    Returns:
        [type]: [description]
    """

    m       = 30/100  # Proportion of admitted already colonized individuals with the sensitive strain [adim]
    mu      = 1/15    # Average duration of hospital stay [days]
    gamma   = 1/45    # Average time from admission or colonization until spontaneous clearance of bacteria carriage [days]
    tau     = 1/14    # Average time from admission to colonization

    X       = x[0, :] # Non colonized individuals
    S       = x[1, :] # Colonized individuals with the sensitive strain
    R       = x[2, :] # Colonized individuals with the resistant strain

    # force of infection of sensitive strain
    foi_s = beta * S

    # force of infection of resistant strain
    foi_r = beta * R


    # Transitions
    x2s      =  transition(X, foi_s, dt) # non-colonized to colonized with sensitive strain
    x2r      =  transition(X, foi_r, dt) # non-colonized to colonized with resistant strain

    # Updates
    X  = X - x2s - x2r
    S  = S + x2s
    R  = R + x2r

    return np.maximum(np.array([X, S, R]), 0)